### 학습데이터 만들기

In [1]:
# !pip install mediapipe

In [1]:
import cv2
import mediapipe as mp
import numpy as np

max_num_hands = 2 # 두개

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=max_num_hands,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# Gesture recognition model
file = np.genfromtxt('gesture_train.csv', delimiter=',')
angle = file[:,:-1].astype(np.float32)
label = file[:, -1].astype(np.float32)
knn = cv2.ml.KNearest_create()
knn.train(angle, cv2.ml.ROW_SAMPLE, label)


True

In [2]:
angle_data = np.empty((1,15), int)
angle_data.shape

(1, 15)

In [3]:
import pandas as pd

def make_data(word) :
    angle_data = np.empty((1,15), int)
    
    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, img = cap.read()
        if not ret:
            continue

        img = cv2.flip(img, 1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        result = hands.process(img)

        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if result.multi_hand_landmarks is not None:
            rps_result = []

            for res in result.multi_hand_landmarks:
                joint = np.zeros((21, 3))
                for j, lm in enumerate(res.landmark):
                    joint[j] = [lm.x, lm.y, lm.z]

                # Compute angles between joints
                v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint
                v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint
                v = v2 - v1 # [20,3]
                
                # Normalize v 정규화
                v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                # Get angle using arcos of dot product 아크코사인으로 15개의 각도를 저장
                angle = np.arccos(np.einsum('nt,nt->n',
                    v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                    v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

                angle = np.degrees(angle) # Convert radian to degree

                # Inference gesture
                data = np.array([angle], dtype=np.float32)

                angle_data = np.append(angle_data, data, axis=0)

                mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

        cv2.imshow('make_train_data', img)
        if cv2.waitKey(1) == ord('q'):
            cv2.destroyAllWindows()
            cap.release()
            df = pd.DataFrame(angle_data)
            df['word'] = word
            #df.columns = ['00','1','2','3','4','5','6','7','8','9','10','11','12','13','14','wor']
            return df;

In [4]:
df1 = make_data('100') # 오른손
df1.shape

(203, 16)

In [8]:
df2 = make_data('101') # 왼손
df2.shape

(504, 16)

In [9]:
df = pd.merge(df1, df2, how='outer')
#df

In [10]:
# 파일 저장
df.to_csv('word_add.csv', index = False)

### 기존 데이터셋과 합치기

In [ ]:
import pandas as pd

ori_data = pd.read_csv('word.csv')
# ori_data.info()

In [ ]:
data = pd.read_csv('word_add.csv')
data.info()

In [ ]:
df = pd.merge(ori_data, data, how='outer')
df.info()

In [ ]:
df.to_csv('word.csv', index = False)